# Public Assistance Funded Project Summaries
Author: Mark Bauer

In [1]:
# import libraries
import duckdb
from datetime import datetime

In [2]:
# reproducibility
%reload_ext watermark
%watermark -v -p duckdb

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.6.0

duckdb: 1.0.0



In [3]:
# data retrieved
current_date = datetime.now()
print(f"This notebook was executed on {current_date.strftime('%Y-%m-%d')}.")

This notebook was executed on 2024-10-03.


# OpenFEMA Dataset: Public Assistance Funded Project Summaries - v1

## Dataset Description
>FEMA provides supplemental Federal disaster grant assistance for debris removal, emergency protective measures, and the repair, replacement, or restoration of disaster-damaged, publicly owned facilities and the facilities of certain Private Non-Profit (PNP) organizations through the Public Assistance (PA) Program (CDFA Number 97.036). The PA Program also encourages protection of these damaged facilities from future events by providing assistance for hazard mitigation measures during the recovery process (see FEMA Hazard Mitigation Grants Sections 404 and 406: https://www.fema.gov/press-release/20230426/fema-hazard-mitigation-grants-404-and-406).

Source: [OpenFEMA Dataset: Public Assistance Funded Project Summaries - v1](https://www.fema.gov/openfema-data-page/public-assistance-funded-project-summaries-v1)

## Disaster Delcarations
For more information on the disaster declaration process:
- Information about disasters: https://www.fema.gov/disasters
- How a disaster is declared: https://www.fema.gov/disasters/how-declared

## Citation
Federal Emergency Management Agency (FEMA), OpenFEMA Dataset: Public Assistance Funded Project Summaries - v1. Retrieved from https://www.fema.gov/openfema-data-page/public-assistance-funded-project-summaries-v1. This product uses the FEMA OpenFEMA API, but is not endorsed by FEMA. The Federal Government or FEMA cannot vouch for the data or analyses derived from these data after the data have been retrieved from the Agency's website(s).

Read more about [OpenFEMA Terms and Conditions](https://www.fema.gov/about/openfema/terms-conditions).

# Additional Resources
To examine other cool data visualizations about this data, visit FEMA's [Public Assistance Program Summary of Obligations](https://www.fema.gov/data-visualization/public-assistance-program-summary-obligations) page.

# Read In Data

In [4]:
# duckdb connection
con = duckdb.connect()

# create public assistance table
con.execute("""
    CREATE TABLE public_assistance AS
    FROM read_parquet('https://www.fema.gov/api/open/v1/PublicAssistanceFundedProjectsSummaries.parquet')
"""
)

# sanity check
sql = """
    SELECT *
    FROM public_assistance
    LIMIT 10
"""

con.sql(sql)

┌──────────────────────┬────────────────┬─────────────────┬───┬──────────────────────┬──────────────────────┐
│          id          │ disasterNumber │ declarationDate │ … │     lastRefresh      │         hash         │
│       varchar        │     int16      │      date       │   │      timestamp       │       varchar        │
├──────────────────────┼────────────────┼─────────────────┼───┼──────────────────────┼──────────────────────┤
│ 0646e381-ea0a-46e5…  │           1239 │ 1998-08-26      │ … │ 2023-03-18 17:02:4…  │ a18872962143d707fb…  │
│ bc05fe25-0f1e-4344…  │           1239 │ 1998-08-26      │ … │ 2023-03-18 17:02:4…  │ d1c77a224203725a79…  │
│ d9c845c8-5db4-404b…  │           1239 │ 1998-08-26      │ … │ 2023-03-18 17:02:4…  │ 7fb2d6dd706512236b…  │
│ 9350e760-64da-4931…  │           1239 │ 1998-08-26      │ … │ 2023-03-18 17:02:4…  │ 571fab79609fe373f6…  │
│ a196ef51-9883-47c8…  │           1239 │ 1998-08-26      │ … │ 2023-03-18 17:02:4…  │ 39207f1c57e1f088be…  │
│ 39de6538

# Describe and Summarize Data

In [5]:
# count rows
con.sql("SELECT COUNT(id) AS count_rows FROM public_assistance")

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│     185652 │
└────────────┘

In [6]:
# examine column datatypes
sql = "DESCRIBE SELECT * FROM public_assistance"
describe_relation = con.sql(sql)

(con
 .sql("SELECT column_name, column_type FROM describe_relation")
 .show(max_rows=50)
)

┌────────────────────────┬─────────────┐
│      column_name       │ column_type │
│        varchar         │   varchar   │
├────────────────────────┼─────────────┤
│ id                     │ VARCHAR     │
│ disasterNumber         │ SMALLINT    │
│ declarationDate        │ DATE        │
│ incidentType           │ VARCHAR     │
│ state                  │ VARCHAR     │
│ county                 │ VARCHAR     │
│ applicantName          │ VARCHAR     │
│ educationApplicant     │ BOOLEAN     │
│ numberOfProjects       │ SMALLINT    │
│ federalObligatedAmount │ DOUBLE      │
│ lastRefresh            │ TIMESTAMP   │
│ hash                   │ VARCHAR     │
├────────────────────────┴─────────────┤
│ 12 rows                    2 columns │
└──────────────────────────────────────┘



In [7]:
# summary statistics
sql = "SUMMARIZE SELECT * FROM public_assistance"
summarize_relation = con.sql(sql)

summarize_relation.df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,id,VARCHAR,00002220-90a3-474a-a2fc-5cd033195b36,ffff6434-3cc9-4130-8363-32de43bddcb8,187613,None,None,None,None,None,185652,0.0
1,disasterNumber,SMALLINT,1239,4812,1648,2909.3991661818886,1259.0059165904195,1691,3150,4245,185652,0.0
2,declarationDate,DATE,1998-08-26,2024-08-20,1264,None,None,None,None,None,185652,0.0
3,incidentType,VARCHAR,Biological,Winter Storm,28,None,None,None,None,None,185652,0.0
4,state,VARCHAR,Alabama,Wyoming,63,None,None,None,None,None,185652,0.0
5,county,VARCHAR,,Ziebach,3554,None,None,None,None,None,185652,0.0
6,applicantName,VARCHAR,#9 AREA FIRE DEPARTMENT,plainville,87293,None,None,None,None,None,185652,0.0
7,educationApplicant,BOOLEAN,false,true,2,None,None,None,None,None,185652,0.0
8,numberOfProjects,SMALLINT,0,2685,328,4.482418718893413,17.361197327824826,1,2,4,185652,0.0
9,federalObligatedAmount,DOUBLE,-1850352.29,12954204261.16,174805,1276227.0047087406,53117208.56566425,6687.23022716714,23306.762525691047,96196.14572380968,185652,0.0


In [8]:
# examine NULL percentage
sql = """
    SELECT column_name, column_type, null_percentage
    FROM summarize_relation
    ORDER BY null_percentage DESC
"""

con.sql(sql).show(max_rows=50)

┌────────────────────────┬─────────────┬─────────────────┐
│      column_name       │ column_type │ null_percentage │
│        varchar         │   varchar   │  decimal(9,2)   │
├────────────────────────┼─────────────┼─────────────────┤
│ id                     │ VARCHAR     │            0.00 │
│ disasterNumber         │ SMALLINT    │            0.00 │
│ declarationDate        │ DATE        │            0.00 │
│ incidentType           │ VARCHAR     │            0.00 │
│ state                  │ VARCHAR     │            0.00 │
│ county                 │ VARCHAR     │            0.00 │
│ applicantName          │ VARCHAR     │            0.00 │
│ educationApplicant     │ BOOLEAN     │            0.00 │
│ numberOfProjects       │ SMALLINT    │            0.00 │
│ federalObligatedAmount │ DOUBLE      │            0.00 │
│ lastRefresh            │ TIMESTAMP   │            0.00 │
│ hash                   │ VARCHAR     │            0.00 │
├────────────────────────┴─────────────┴────────────────

# Preview Values

In [9]:
sql = """
    SELECT
        *
    FROM
        public_assistance
    LIMIT 5   
"""

con.sql(sql).df()

,id,disasterNumber,declarationDate,incidentType,state,county,applicantName,educationApplicant,numberOfProjects,federalObligatedAmount,lastRefresh,hash
0,0646e381-ea0a-46e5-993d-1bdb1cfe2a27,1239,1998-08-26,Severe Storm,Texas,Edwards,EDWARDS (COUNTY),False,20,352427.09,2023-03-18 17:02:48.443,a18872962143d707fb83e3f0b054abda40413d04
1,bc05fe25-0f1e-4344-a120-55f23f45cd4e,1239,1998-08-26,Severe Storm,Texas,Kimble,"JUNCTION, CITY OF",False,1,6234.90,2023-03-18 17:02:48.443,d1c77a224203725a79d26a1d84f8ad2b06b36c7f
2,d9c845c8-5db4-404b-866d-e06b3c73397a,1239,1998-08-26,Severe Storm,Texas,Kimble,KIMBLE (COUNTY),False,5,69933.35,2023-03-18 17:02:48.443,7fb2d6dd706512236b3ed6c0e6ddd6a633b50d38
3,9350e760-64da-4931-ace0-0e37d7e82eb3,1239,1998-08-26,Severe Storm,Texas,Kimble,TEXAS TECH UNIVERSITY CTR,True,1,2850.00,2023-03-18 17:02:48.443,571fab79609fe373f64529b540289ee5dd400ed5
4,a196ef51-9883-47c8-809d-1d797e8e1f48,1239,1998-08-26,Severe Storm,Texas,Kinney,FORT CLARK MUNICIPAL UTILITY,False,1,5016.00,2023-03-18 17:02:48.443,39207f1c57e1f088bef0ce07238c1aa9a18022d8


In [10]:
# count per declaration and applicant
sql = """
    SELECT
       disasterNumber,
       applicantName,
       count(id) AS count
    FROM
        public_assistance
    GROUP BY
        1, 2
    ORDER BY
        count DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY count DESC
    LIMIT 20
""")

┌────────────────┬──────────────────────────┬───────┐
│ disasterNumber │      applicantName       │ count │
│     int16      │         varchar          │ int64 │
├────────────────┼──────────────────────────┼───────┤
│           1580 │ JACKSON, TOWNSHIP OF     │    17 │
│           1580 │ WASHINGTON, TOWNSHIP OF  │    16 │
│           3180 │ WASHINGTON (TOWNSHIP OF) │    13 │
│           1453 │ JEFFERSON, TOWNSHIP OF   │    12 │
│           1557 │ WASHINGTON (TOWNSHIP OF) │    12 │
│           1580 │ UNION, TOWNSHIP OF       │    12 │
│           1453 │ WASHINGTON, TOWNSHIP OF  │    11 │
│           1580 │ MONROE, TOWNSHIP OF      │    11 │
│           1580 │ PERRY, TOWNSHIP OF       │    10 │
│           1453 │ JACKSON, TOWNSHIP OF     │    10 │
│           1453 │ PERRY, TOWNSHIP OF       │     9 │
│           1557 │ JACKSON (TOWNSHIP OF)    │     9 │
│           1898 │ WASHINGTON (TOWNSHIP OF) │     9 │
│           1580 │ SALEM, TOWNSHIP OF       │     9 │
│           3180 │ PENN (TOW

The data suggests that one Disaster Number and Applicant can have multiple records in this dataset. Let's examine further.

In [11]:
# examine count unique for applicantName = 'JACKSON, TOWNSHIP OF'
(con
 .sql("""
    SELECT *
    FROM public_assistance
    WHERE applicantName = 'JACKSON, TOWNSHIP OF'
        AND disasterNumber = 1580
""")
 .df()
 .describe(include='object')
 .loc['unique']
)

id               17
incidentType      1
state             1
county           17
applicantName     1
hash             17
Name: unique, dtype: object

Data suggests that an Applicant can have many counties. Let's group by county to confirm.

In [12]:
# group by columns and order by DESC to view unique counts
sql = """
    SELECT
        disasterNumber,
        applicantName,
        county,
        count(id) AS count
    FROM
        public_assistance
    GROUP BY
        1, 2, 3
    ORDER BY
        count DESC   
"""

relation = con.sql(sql)

# use 'JACKSON, TOWNSHIP OF' as example
con.sql("""
    SELECT *
    FROM relation
    WHERE applicantName = 'JACKSON, TOWNSHIP OF'
        AND disasterNumber = 1580
    ORDER BY count DESC
""")

┌────────────────┬──────────────────────┬────────────┬───────┐
│ disasterNumber │    applicantName     │   county   │ count │
│     int16      │       varchar        │  varchar   │ int64 │
├────────────────┼──────────────────────┼────────────┼───────┤
│           1580 │ JACKSON, TOWNSHIP OF │ Crawford   │     1 │
│           1580 │ JACKSON, TOWNSHIP OF │ Franklin   │     1 │
│           1580 │ JACKSON, TOWNSHIP OF │ Highland   │     1 │
│           1580 │ JACKSON, TOWNSHIP OF │ Shelby     │     1 │
│           1580 │ JACKSON, TOWNSHIP OF │ Noble      │     1 │
│           1580 │ JACKSON, TOWNSHIP OF │ Allen      │     1 │
│           1580 │ JACKSON, TOWNSHIP OF │ Pickaway   │     1 │
│           1580 │ JACKSON, TOWNSHIP OF │ Hardin     │     1 │
│           1580 │ JACKSON, TOWNSHIP OF │ Darke      │     1 │
│           1580 │ JACKSON, TOWNSHIP OF │ Knox       │     1 │
│           1580 │ JACKSON, TOWNSHIP OF │ Monroe     │     1 │
│           1580 │ JACKSON, TOWNSHIP OF │ Guernsey   │ 

From the analysis above, unique record consists of a disaster, an Applicant, and a county.

# Analysis

In [13]:
sql = """
    SELECT
        ROUND(SUM(federalObligatedAmount), 0)::BIGINT AS federalObligatedAmount
    FROM
        public_assistance  
"""

con.sql(sql)

┌────────────────────────┐
│ federalObligatedAmount │
│         int64          │
├────────────────────────┤
│           236934095878 │
└────────────────────────┘

In [14]:
# format dollar value
federalObligatedAmount = con.sql(sql).df().values[0][0]

print(f"Total federal obligated amount: ${federalObligatedAmount:,}")

Total federal obligated amount: $236,934,095,878


In [15]:
sql = """
    SELECT
        COUNT(DISTINCT disasterNumber)
    FROM
        public_assistance
"""

disasterNumber = con.sql(sql).df().values[0][0]

print(f"Total disaster declarations: {disasterNumber:,}")

Total disaster declarations: 1,651


In [16]:
sql = """
    SELECT
        COUNT(DISTINCT applicantName)
    FROM
        public_assistance  
    WHERE 
        numberOfProjects > 0
"""

applicantName = con.sql(sql).df().values[0][0]

print(f"Number of applicants with funded projects: {applicantName:,}")

Number of applicants with funded projects: 86,167


In [17]:
sql = """
    SELECT
        SUM(numberOfProjects)::BIGINT
    FROM
        public_assistance  
        
"""

numberOfProjects = con.sql(sql).df().values[0][0]

print(f"Total funded Project Worksheets (PWs): {numberOfProjects:,}")

Total funded Project Worksheets (PWs): 832,170


**Table xx.** Top 20 Disaster Declarations and States with the Highest Federal Obligated Amount

In [18]:
sql = """
    SELECT
        disasterNumber,
        state,
        ROUND(SUM(federalObligatedAmount), 0)::BIGINT AS federalObligatedAmount
    FROM
        public_assistance
    GROUP BY
        disasterNumber, state
    ORDER BY
        federalObligatedAmount DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY federalObligatedAmount DESC
    LIMIT 20
""")

┌────────────────┬────────────────┬────────────────────────┐
│ disasterNumber │     state      │ federalObligatedAmount │
│     int16      │    varchar     │         int64          │
├────────────────┼────────────────┼────────────────────────┤
│           4339 │ Puerto Rico    │            33962164412 │
│           4480 │ New York       │            15056437208 │
│           4485 │ Texas          │            14909743427 │
│           4085 │ New York       │            14735635709 │
│           1603 │ Louisiana      │            13524620378 │
│           4340 │ Virgin Islands │            12239518967 │
│           4482 │ California     │            10609269973 │
│           1391 │ New York       │             4708614487 │
│           1604 │ Mississippi    │             3184451567 │
│           4486 │ Florida        │             3051885056 │
│           4496 │ Massachusetts  │             2975723414 │
│           4488 │ New Jersey     │             2960234354 │
│           4332 │ Texas

**Table xx.** Top 20 Disaster Declarations and States with the Highest Federal Obligated Amount

In [19]:
sql = """
    SELECT
        disasterNumber,
        applicantName,
        ROUND(SUM(federalObligatedAmount), 0)::BIGINT AS federalObligatedAmount
    FROM
        public_assistance
    GROUP BY
        disasterNumber, applicantName
    ORDER BY
        federalObligatedAmount DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY federalObligatedAmount DESC
    LIMIT 20
""")

┌────────────────┬─────────────────────────────────────────────────────────────┬────────────────────────┐
│ disasterNumber │                        applicantName                        │ federalObligatedAmount │
│     int16      │                           varchar                           │         int64          │
├────────────────┼─────────────────────────────────────────────────────────────┼────────────────────────┤
│           4339 │ PR Electric Power Authority                                 │            12954204261 │
│           4485 │ Texas Department of State Health Services                   │             9697013440 │
│           4480 │ City of New York - Management and Budget                    │             8613685564 │
│           4085 │ NEW YORK                                                    │             8513455550 │
│           4339 │ PR Aqueduct and Sewer Authority                             │             4627455402 │
│           4485 │ Texas Division of Emergency

**Table xx.** Top 20 Disaster Declarations and Applicants with the Highest Federal Obligated Amount

In [20]:
sql = """
    SELECT
        substring(declarationDate::VARCHAR, 1, 4) as year,
        ROUND(SUM(federalObligatedAmount), 0)::BIGINT AS federalObligatedAmount
    FROM
        public_assistance
    GROUP BY
        year
    ORDER BY
        federalObligatedAmount DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY federalObligatedAmount DESC
    LIMIT 20
""")

┌─────────┬────────────────────────┐
│  year   │ federalObligatedAmount │
│ varchar │         int64          │
├─────────┼────────────────────────┤
│ 2020    │            88375733472 │
│ 2017    │            54170092744 │
│ 2005    │            21256945361 │
│ 2012    │            17783759641 │
│ 2018    │             6305405051 │
│ 2008    │             5945566626 │
│ 2001    │             5843745213 │
│ 2021    │             5138792833 │
│ 2022    │             4601446539 │
│ 2011    │             3819025442 │
│ 2004    │             3400392826 │
│ 2016    │             2799028865 │
│ 2019    │             2282604905 │
│ 2023    │             1740736615 │
│ 2007    │             1711103631 │
│ 2009    │             1605743093 │
│ 2010    │             1533523763 │
│ 2013    │             1243436633 │
│ 2015    │             1167517310 │
│ 2006    │             1087690898 │
├─────────┴────────────────────────┤
│ 20 rows                2 columns │
└──────────────────────────────────┘

**Table xx.** Top 20 Years with the Highest Federal Obligated Amount

In [21]:
sql = """
    SELECT
        incidentType,
        ROUND(SUM(federalObligatedAmount), 0)::BIGINT AS federalObligatedAmount
    FROM
        public_assistance
    GROUP BY
        incidentType
    ORDER BY
        federalObligatedAmount DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY federalObligatedAmount DESC
""").show(max_rows=50)

┌─────────────────────────────────┬────────────────────────┐
│          incidentType           │ federalObligatedAmount │
│             varchar             │         int64          │
├─────────────────────────────────┼────────────────────────┤
│ Hurricane                       │           113355465305 │
│ Biological                      │            81752672809 │
│ Severe Storm                    │            13092177707 │
│ Fire                            │             6398730254 │
│ Flood                           │             5370081579 │
│ Wildfire                        │             3889841629 │
│ Severe Storm(s)                 │             3639790895 │
│ Severe Ice Storm                │             1906764889 │
│ Earthquake                      │             1503749541 │
│ Tropical Storm                  │             1463069897 │
│ Snowstorm                       │             1190088584 │
│ Winter Storm                    │              822070100 │
│ Tornado               

**Table xx.** Incident Types with the Highest Federal Obligated Amount

In [22]:
sql = """
    SELECT
        substring(declarationDate::VARCHAR, 1, 4) as year,
        incidentType,
        ROUND(SUM(federalObligatedAmount), 0)::BIGINT AS federalObligatedAmount
    FROM
        public_assistance
    GROUP BY
        year, incidentType
    ORDER BY
        federalObligatedAmount DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY federalObligatedAmount DESC
    LIMIT 20
""")

┌─────────┬─────────────────┬────────────────────────┐
│  year   │  incidentType   │ federalObligatedAmount │
│ varchar │     varchar     │         int64          │
├─────────┼─────────────────┼────────────────────────┤
│ 2020    │ Biological      │            81749779368 │
│ 2017    │ Hurricane       │            51702067349 │
│ 2005    │ Hurricane       │            20437583611 │
│ 2012    │ Hurricane       │            17384849682 │
│ 2001    │ Fire            │             4795549820 │
│ 2022    │ Hurricane       │             3509237067 │
│ 2020    │ Hurricane       │             3428794938 │
│ 2008    │ Hurricane       │             3293933139 │
│ 2018    │ Hurricane       │             3276880982 │
│ 2004    │ Hurricane       │             2883976508 │
│ 2021    │ Hurricane       │             2841558185 │
│ 2008    │ Severe Storm    │             2248703906 │
│ 2018    │ Wildfire        │             1949261263 │
│ 2019    │ Severe Storm(s) │             1415327401 │
│ 2011    

**Table xx.** Top 20 Years and Incident Types with the Highest Federal Obligated Amount

In [23]:
sql = """
    SELECT
        state,
        ROUND(SUM(federalObligatedAmount), 0)::BIGINT AS federalObligatedAmount
    FROM
        public_assistance
    GROUP BY
        state
    ORDER BY
        federalObligatedAmount DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY federalObligatedAmount DESC
    LIMIT 20
""")

┌────────────────┬────────────────────────┐
│     state      │ federalObligatedAmount │
│    varchar     │         int64          │
├────────────────┼────────────────────────┤
│ New York       │            37470703059 │
│ Puerto Rico    │            36804567516 │
│ Louisiana      │            24702773099 │
│ Texas          │            22668021776 │
│ California     │            17002685086 │
│ Florida        │            16507361698 │
│ Virgin Islands │            12465714202 │
│ New Jersey     │             6064328491 │
│ Mississippi    │             4233026069 │
│ North Carolina │             4013296086 │
│ Massachusetts  │             3701976396 │
│ Washington     │             3217424675 │
│ Colorado       │             2759097871 │
│ Maryland       │             2459344288 │
│ Illinois       │             2440749134 │
│ Iowa           │             2380877862 │
│ Pennsylvania   │             2241351498 │
│ Virginia       │             1830251778 │
│ Oregon         │             1

**Table xx.** Top 20 States with the Highest Federal Obligated Amount

In [24]:
sql = """
    SELECT
        state,
        county,
        ROUND(SUM(federalObligatedAmount), 0)::BIGINT AS federalObligatedAmount
    FROM
        public_assistance
    GROUP BY
        state, county
    ORDER BY
        federalObligatedAmount DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY federalObligatedAmount DESC
    LIMIT 20
""")

┌────────────────┬────────────────────┬────────────────────────┐
│     state      │       county       │ federalObligatedAmount │
│    varchar     │      varchar       │         int64          │
├────────────────┼────────────────────┼────────────────────────┤
│ Puerto Rico    │ Statewide          │            29119979157 │
│ Texas          │ Statewide          │            16410713752 │
│ New York       │ New York           │            12531417444 │
│ Virgin Islands │ Statewide          │            12430079292 │
│ California     │ Statewide          │            10171603998 │
│ New York       │ New York County    │             9735811199 │
│ Louisiana      │ Statewide          │             8730357357 │
│ New York       │ Statewide          │             7412039715 │
│ Florida        │ Statewide          │             6287064559 │
│ Louisiana      │ Orleans            │             5064821871 │
│ New Jersey     │ Statewide          │             2822501914 │
│ Colorado       │ Statew

**Table xx.** Top 20 States and Counties with the Highest Federal Obligated Amount

In [25]:
sql = """
    SELECT
        state,
        incidentType,
        ROUND(SUM(federalObligatedAmount), 0)::BIGINT AS federalObligatedAmount
    FROM
        public_assistance
    GROUP BY
        state, incidentType
    ORDER BY
        federalObligatedAmount DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY federalObligatedAmount DESC
    LIMIT 20
""")

┌────────────────┬──────────────┬────────────────────────┐
│     state      │ incidentType │ federalObligatedAmount │
│    varchar     │   varchar    │         int64          │
├────────────────┼──────────────┼────────────────────────┤
│ Puerto Rico    │ Hurricane    │            35547364578 │
│ Louisiana      │ Hurricane    │            21112502682 │
│ New York       │ Hurricane    │            15342097227 │
│ New York       │ Biological   │            15059999395 │
│ Texas          │ Biological   │            14909743427 │
│ Florida        │ Hurricane    │            12636561392 │
│ Virgin Islands │ Hurricane    │            12306903721 │
│ California     │ Biological   │            10618638278 │
│ Texas          │ Hurricane    │             6175842977 │
│ New York       │ Fire         │             4708614487 │
│ Mississippi    │ Hurricane    │             3297426051 │
│ California     │ Wildfire     │             3057389795 │
│ Florida        │ Biological   │             3053694701

**Table xx.** Top 20 Disaster Declarations, States, Years, and Incident Types with the Highest Federal Obligated Amount

In [26]:
sql = """
    SELECT
        disasterNumber,
        state,
        substring(declarationDate::VARCHAR, 1, 4) as year,
        incidentType,
        ROUND(SUM(federalObligatedAmount), 0)::BIGINT AS federalObligatedAmount
    FROM
        public_assistance
    GROUP BY
        disasterNumber, state, year, incidentType
    ORDER BY
        federalObligatedAmount DESC   
"""

relation = con.sql(sql)

con.sql("""
    SELECT *
    FROM relation
    ORDER BY federalObligatedAmount DESC
    LIMIT 20
""")

┌────────────────┬────────────────┬─────────┬──────────────┬────────────────────────┐
│ disasterNumber │     state      │  year   │ incidentType │ federalObligatedAmount │
│     int16      │    varchar     │ varchar │   varchar    │         int64          │
├────────────────┼────────────────┼─────────┼──────────────┼────────────────────────┤
│           4339 │ Puerto Rico    │ 2017    │ Hurricane    │            33962164412 │
│           4480 │ New York       │ 2020    │ Biological   │            15056437208 │
│           4485 │ Texas          │ 2020    │ Biological   │            14909743427 │
│           4085 │ New York       │ 2012    │ Hurricane    │            14735635709 │
│           1603 │ Louisiana      │ 2005    │ Hurricane    │            13524620378 │
│           4340 │ Virgin Islands │ 2017    │ Hurricane    │            12239518967 │
│           4482 │ California     │ 2020    │ Biological   │            10609269973 │
│           1391 │ New York       │ 2001    │ Fire    